# PLOG replace

This notebook shows how to use the code to read a chemkin kinetic model and simply replacing each of the PLOG reaction with a single modified arrhenius expression at a desired Pressure level.
The code automatically recognize the PLOG reactions and when the PLOG are defined implicitly duplicate it splits them in two different reactions.

In [1]:
import os
import sys

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

from diffPLOG2TROE.pressure_logarithmic import kinetic_constant_plog
from diffPLOG2TROE.chemkin.interpreter import read_chemkin_extract_plog
from diffPLOG2TROE.chemkin.writer import write_chemkin
from diffPLOG2TROE.refitter import refit_plog

In [2]:
current_file_dir = os.getcwd()
module_path = os.path.abspath(os.path.join(current_file_dir, ".."))
mechanism = os.path.join(module_path, "examples", "data", "CRECK_C1C3_HT.CKI")
plog_reactions, idx_plog, idx_reactions = read_chemkin_extract_plog(
    kinetics=mechanism,
)

 Reading the CHEMKIN kinetic file: /Users/tdinelli/Documents/GitHub/diffPLOG2TROE/examples/data/CRECK_C1C3_HT.CKI
 * Total number of reactions: 1999
    - Number of PLOG reactions: 188


In [3]:
TARGET_PRESSURE = 4.5

converted_plog_reactions = []
fitting_parameters = []
for i in plog_reactions:
    tmp = {
        "name": "",
        "parameters": [],
        "is_duplicate": False
    }
    if i["is_duplicate"] is True:
        tmp["is_duplicate"] = True

    print("Reaction: {}".format(i["name"]))
    
    jax_plog = jnp.asarray(i["parameters"], dtype=jnp.float64)
    A, b, Ea, R2adj, fg = refit_plog(jax_plog, TARGET_PRESSURE)
    
    print(f" A: {A:.5E} b: {b:.5E} Ea: {Ea:.5E}\n Adjusted R2: {R2adj:.5E}")
    tmp["name"] = i["name"]
    tmp["parameters"] = [float(A), float(b), float(Ea)]
    converted_plog_reactions.append(tmp)
    fg = jnp.append(fg, R2adj)
    fitting_parameters.append(fg)

Reaction: HOCO=OH+CO
 A: 9.16672E+28 b: -5.57865E+00 Ea: 2.85350E+04
 Adjusted R2: 9.99997E-01
Reaction: HOCO=H+CO2
 A: 1.80310E+36 b: -8.08998E+00 Ea: 2.93360E+04
 Adjusted R2: 9.99999E-01
Reaction: OH+CH3=H2O+CH2(S)
 A: 1.80615E+16 b: -1.07225E+00 Ea: 1.77202E+03
 Adjusted R2: 6.66729E-01
Reaction: OH+CH3=H2+CH2O
 A: 3.26230E+06 b: 1.18815E+00 Ea: -2.00999E+03
 Adjusted R2: 7.65680E-01
Reaction: OH+CH3=H+CH2OH
 A: 1.53749E+10 b: 9.77390E-01 Ea: 3.56601E+03
 Adjusted R2: 9.97425E-01
Reaction: OH+CH3=H+CH3O
 A: 1.21605E+09 b: 1.01281E+00 Ea: 1.19500E+04
 Adjusted R2: 9.99999E-01
Reaction: 2CH3=H+C2H5
 A: 4.08198E+11 b: 4.98886E-01 Ea: 1.33725E+04
 Adjusted R2: 9.99998E-01
Reaction: O2+C2H5=C2H5O2
 A: 1.01801E+58 b: -1.44591E+01 Ea: 1.42400E+04
 Adjusted R2: 9.99764E-01
Reaction: O2+C2H5=C2H4O2H
 A: 1.52200E+34 b: -8.39454E+00 Ea: 7.71000E+03
 Adjusted R2: 9.92669E-01
Reaction: O2+C2H5=HO2+C2H4
 A: 4.33134E+06 b: 1.31630E+00 Ea: -7.20592E+02
 Adjusted R2: 8.86702E-01
Reaction: O2+C2H5=O

In [4]:
output_folder = os.path.join(module_path, "examples", "data")
write_chemkin(
    kinetics=mechanism,
    output_folder=output_folder,
    plog_converted=converted_plog_reactions,
    fitting_parameters = fitting_parameters
)